In [1]:
"""
Submission by Yijie Xu

Section 1. Preprocessing via ImageAI & Yolo detection by cropping background to improve classification accuracy via removal of distractions

Pipeline

1. Install ImageAI
2. Define supporting classes for filtering purposes 
3. Initialize YoloV3 detector for object cropping
4. Download pre-segregated Stanford dataset
5. Define directories for inputs and outputs (will assist during classification as well)
6. Run detector on all images

Once done, performed, cropped dataset was saved and uploaded to personal gdrive for Section 2 - MobileNet retraining

"""


!pip3 install https://github.com/OlafenwaMoses/ImageAI/releases/download/2.0.2/imageai-2.0.2-py3-none-any.whl



from imageai.Detection import ObjectDetection
import os
import sys
sys.path.append("..")

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img, save_img
import numpy as np


import numpy as np

def filter_files(files_list, filter_by = None):
    try:
        files_list.remove('.DS_Store')
    except:
        x=1
        
    if filter_by is not None:
        mask = np.logical_not(np.isin(files_list, filter_by))
        result = []
        for f, m in zip(files_list, mask):
            if m == True:
                result.append(f)

        return result
    else:
        return files_list
      

from imageai.Detection import ObjectDetection
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img, save_img
import os
from matplotlib import pyplot as plt
import numpy as np


class Detector:
    def __init__(self, model):
        execution_path = os.getcwd()
        detector = ObjectDetection()

        if model == 'yolo':
            detector.setModelTypeAsYOLOv3()
        elif model == 'yolo-tiny':
            detector.setModelTypeAsTinyYOLOv3()
        else:
            raise ValueError('Model ' + model + 'not fould. you should download the model and put it into "modules" directory.')

        detector.setModelPath(os.path.join(execution_path , '../content/' + model + '.h5'))
        detector.loadModel()
        custom_objects = detector.CustomObjects(car=True)

        self.detector = detector
        self.custom_objects = custom_objects
        self.execution_path = execution_path

    def get_box(self, detections, size, params = {'detect_largest_box': False, 'smallest_detected_area': None}):
        detect_largest_box = params['detect_largest_box']
        smallest_detected_area = params['smallest_detected_area']
                
        area = 0
        points = None

        if detect_largest_box == True:
            for detection in detections:
                current_area = (detection['box_points'][2] - detection['box_points'][0]) * (detection['box_points'][3] - detection['box_points'][1])
                if current_area > area:
                    area = current_area
                    points = detection['box_points']
        else:
            points = detections[0]['box_points']

        img_area = size[0] * size[1] if smallest_detected_area != None else None
        
        if (img_area != None) and (area / img_area < smallest_detected_area):
            return None
        else:
            return points

    def crop_image(self, source_image, input_type, **kwargs):
        image = None
        input_image = None
        if input_type == 'file':
            image = load_img(source_image)
            input_image = source_image
        elif input_type == 'array':
            image = source_image
            input_image = img_to_array(source_image)

        detections = self.detector.detectCustomObjectsFromImage(
            custom_objects=self.custom_objects,
            input_type=input_type,
            input_image=input_image, 
            minimum_percentage_probability=10,
            extract_detected_objects=False,
        )

        box = self.get_box(detections, image.size, **kwargs)
        cropped_img = image.crop(box)

        return cropped_img

    def crop_dir(self, source_path, target_path, directory, **kwargs):
        if os.path.exists(target_path + directory) == False:
            os.mkdir(target_path + directory)
        
        filenames = os.listdir(source_path + directory)
        filenames = filter_files(filenames)
        
        for filename in filenames:
            image_path = os.path.join(self.execution_path , source_path + directory + '/' + filename)
            cropped_img = self.crop_image(image_path, 'file', **kwargs)
            save_img(target_path + directory + '/' + filename, cropped_img)
        
    def crop_path(self, source_path, target_path, dirs, **kwargs):
        target_dirs = os.listdir(target_path)
        dirs = filter_files(dirs, target_dirs)

        if os.path.exists(target_path) == False:
            os.mkdir(target_path)
        
        for i, directory in enumerate(dirs):
            print('Directory', str(i), 'of', str(len(dirs)), '...')
            self.crop_dir(source_path, target_path, directory, **kwargs)
            print('Images in directory', str(i), '"' + directory + '"', 'cropped')


     |████████████████████████████████| 153kB 236kB/s 


Using TensorFlow backend.


In [2]:
#Download a basic Yolo detector
!gdown https://drive.google.com/uc?id=1caedWZlBXHXriD0-QeH8-Ihz2Rlq_DOV
  
#Download stanford car dataset- preseparated into classes

!gdown https://drive.google.com/uc?id=1a_3hAT9Jr7jtWrJ5HSh5q6HlK0P8GUZa

Downloading...
From: https://drive.google.com/uc?id=1caedWZlBXHXriD0-QeH8-Ihz2Rlq_DOV
To: /content/yolo.h5
249MB [00:09, 25.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1a_3hAT9Jr7jtWrJ5HSh5q6HlK0P8GUZa
To: /content/car_data.zip
1.96GB [00:30, 65.3MB/s]


In [3]:
#Master list of all classes of images


dirs = [ 'Audi 100 Sedan 1994',
 'Audi 100 Wagon 1994',
 'Audi A5 Coupe 2012',
 'Audi R8 Coupe 2012',
 'Audi RS 4 Convertible 2008',
 'Audi S4 Sedan 2007',
 'Audi S4 Sedan 2012',
 'Audi S5 Convertible 2012',
 'Audi S5 Coupe 2012',
 'Audi S6 Sedan 2011',
 'Audi TT Hatchback 2011',
 'Audi TT RS Coupe 2012',
 'Audi TTS Coupe 2012',
 'Audi V8 Sedan 1994',
 'Bentley Arnage Sedan 2009',
 'Bentley Continental Flying Spur Sedan 2007',
 'Bentley Continental GT Coupe 2007',
 'Bentley Continental GT Coupe 2012',
 'Bentley Continental Supersports Conv. Convertible 2012',
 'Bentley Mulsanne Sedan 2011',
 'BMW 1 Series Convertible 2012',
 'BMW 1 Series Coupe 2012',
 'BMW 3 Series Sedan 2012',
 'BMW 3 Series Wagon 2012',
 'BMW 6 Series Convertible 2007',
 'BMW ActiveHybrid 5 Sedan 2012',
 'BMW M3 Coupe 2012',
 'BMW M5 Sedan 2010',
 'BMW M6 Convertible 2010',
 'BMW X3 SUV 2012',
 'BMW X5 SUV 2007',
 'BMW X6 SUV 2012',
 'BMW Z4 Convertible 2012',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Bugatti Veyron 16.4 Coupe 2009',
 'Buick Enclave SUV 2012',
 'Buick Rainier SUV 2007',
 'Buick Regal GS 2012',
 'Buick Verano Sedan 2012',
 'Cadillac CTS-V Sedan 2012',
 'Cadillac Escalade EXT Crew Cab 2007',
 'Cadillac SRX SUV 2012',
 'Chevrolet Avalanche Crew Cab 2012',
 'Chevrolet Camaro Convertible 2012',
 'Chevrolet Cobalt SS 2010',
 'Chevrolet Corvette Convertible 2012',
 'Chevrolet Corvette Ron Fellows Edition Z06 2007',
 'Chevrolet Corvette ZR1 2012',
 'Chevrolet Express Cargo Van 2007',
 'Chevrolet Express Van 2007',
 'Chevrolet HHR SS 2010',
 'Chevrolet Impala Sedan 2007',
 'Chevrolet Malibu Hybrid Sedan 2010',
 'Chevrolet Malibu Sedan 2007',
 'Chevrolet Monte Carlo Coupe 2007',
 'Chevrolet Silverado 1500 Classic Extended Cab 2007',
 'Chevrolet Silverado 1500 Extended Cab 2012',
 'Chevrolet Silverado 1500 Hybrid Crew Cab 2012',
 'Chevrolet Silverado 1500 Regular Cab 2012',
 'Chevrolet Silverado 2500HD Regular Cab 2012',
 'Chevrolet Sonic Sedan 2012',
 'Chevrolet Tahoe Hybrid SUV 2012',
 'Chevrolet TrailBlazer SS 2009',
 'Chevrolet Traverse SUV 2012',
 'Chrysler 300 SRT-8 2010',
 'Chrysler Aspen SUV 2009',
 'Chrysler Crossfire Convertible 2008',
 'Chrysler PT Cruiser Convertible 2008',
 'Chrysler Sebring Convertible 2010',
 'Chrysler Town and Country Minivan 2012',
 'Daewoo Nubira Wagon 2002',
 'Dodge Caliber Wagon 2007',
 'Dodge Caliber Wagon 2012',
 'Dodge Caravan Minivan 1997',
 'Dodge Challenger SRT8 2011',
 'Dodge Charger Sedan 2012',
 'Dodge Charger SRT-8 2009',
 'Dodge Dakota Club Cab 2007',
 'Dodge Dakota Crew Cab 2010',
 'Dodge Durango SUV 2007',
 'Dodge Durango SUV 2012',
 'Dodge Journey SUV 2012',
 'Dodge Magnum Wagon 2008',
 'Dodge Ram Pickup 3500 Crew Cab 2010',
 'Dodge Ram Pickup 3500 Quad Cab 2009',
 'Dodge Sprinter Cargo Van 2009',
 'Eagle Talon Hatchback 1998',
 'Ferrari 458 Italia Convertible 2012',
 'Ferrari 458 Italia Coupe 2012',
 'Ferrari California Convertible 2012',
 'Ferrari FF Coupe 2012',
 'FIAT 500 Abarth 2012',
 'FIAT 500 Convertible 2012',
 'Fisker Karma Sedan 2012',
 'Ford Edge SUV 2012',
 'Ford E-Series Wagon Van 2012',
 'Ford Expedition EL SUV 2009',
 'Ford F-150 Regular Cab 2007',
 'Ford F-150 Regular Cab 2012',
 'Ford F-450 Super Duty Crew Cab 2012',
 'Ford Fiesta Sedan 2012',
 'Ford Focus Sedan 2007',
 'Ford Freestar Minivan 2007',
 'Ford GT Coupe 2006',
 'Ford Mustang Convertible 2007',
 'Ford Ranger SuperCab 2011',
 'Geo Metro Convertible 1993',
 'GMC Acadia SUV 2012',
 'GMC Canyon Extended Cab 2012',
 'GMC Savana Van 2012',
 'GMC Terrain SUV 2012',
 'GMC Yukon Hybrid SUV 2012',
 'Honda Accord Coupe 2012',
 'Honda Accord Sedan 2012',
 'Honda Odyssey Minivan 2007',
 'Honda Odyssey Minivan 2012',
 'HUMMER H2 SUT Crew Cab 2009',
 'HUMMER H3T Crew Cab 2010',
 'Hyundai Accent Sedan 2012',
 'Hyundai Azera Sedan 2012',
 'Hyundai Elantra Sedan 2007',
 'Hyundai Elantra Touring Hatchback 2012',
 'Hyundai Genesis Sedan 2012',
 'Hyundai Santa Fe SUV 2012',
 'Hyundai Sonata Hybrid Sedan 2012',
 'Hyundai Sonata Sedan 2012',
 'Hyundai Tucson SUV 2012',
 'Hyundai Veloster Hatchback 2012',
 'Hyundai Veracruz SUV 2012',
 'Infiniti G Coupe IPL 2012',
 'Infiniti QX56 SUV 2011',
 'Isuzu Ascender SUV 2008',
 'Jaguar XK XKR 2012',
 'Jeep Compass SUV 2012',
 'Jeep Grand Cherokee SUV 2012',
 'Jeep Liberty SUV 2012',
 'Jeep Patriot SUV 2012',
 'Jeep Wrangler SUV 2012',
 'Lamborghini Aventador Coupe 2012',
 'Lamborghini Diablo Coupe 2001',
 'Lamborghini Gallardo LP 570-4 Superleggera 2012',
 'Lamborghini Reventon Coupe 2008',
 'Land Rover LR2 SUV 2012',
 'Land Rover Range Rover SUV 2012',
 'Lincoln Town Car Sedan 2011',
 'Maybach Landaulet Convertible 2012',
 'Mazda Tribute SUV 2011',
 'McLaren MP4-12C Coupe 2012',
 'Mercedes-Benz 300-Class Convertible 1993',
 'Mercedes-Benz C-Class Sedan 2012',
 'Mercedes-Benz E-Class Sedan 2012',
 'Mercedes-Benz S-Class Sedan 2012',
 'Mercedes-Benz SL-Class Coupe 2009',
 'Mercedes-Benz Sprinter Van 2012',
 'MINI Cooper Roadster Convertible 2012',
 'Mitsubishi Lancer Sedan 2012',
 'Nissan 240SX Coupe 1998',
 'Nissan Juke Hatchback 2012',
 'Nissan Leaf Hatchback 2012',
 'Nissan NV Passenger Van 2012',
 'Plymouth Neon Coupe 1999',
 'Porsche Panamera Sedan 2012',
 'Ram C-V Cargo Van Minivan 2012',
 'Rolls-Royce Ghost Sedan 2012',
 'Rolls-Royce Phantom Drophead Coupe Convertible 2012',
 'Rolls-Royce Phantom Sedan 2012',
 'Scion xD Hatchback 2012',
 'smart fortwo Convertible 2012',
 'Spyker C8 Convertible 2009',
 'Spyker C8 Coupe 2009',
 'Suzuki Aerio Sedan 2007',
 'Suzuki Kizashi Sedan 2012',
 'Suzuki SX4 Hatchback 2012',
 'Suzuki SX4 Sedan 2012',
 'Tesla Model S Sedan 2012',
 'Toyota 4Runner SUV 2012',
 'Toyota Camry Sedan 2012',
 'Toyota Corolla Sedan 2012',
 'Toyota Sequoia SUV 2012',
 'Volkswagen Beetle Hatchback 2012',
 'Volkswagen Golf Hatchback 1991',
 'Volkswagen Golf Hatchback 2012',
 'Volvo 240 Sedan 1993',
 'Volvo C30 Hatchback 2012',
 'Volvo XC90 SUV 2007',
 'Acura Integra Type R 2001',
 'Acura RL Sedan 2012',
 'Acura TL Sedan 2012',
 'Acura TL Type-S 2008',
 'Acura TSX Sedan 2012',
 'Acura ZDX Hatchback 2012',
 'AM General Hummer SUV 2000',
 'Aston Martin V8 Vantage Convertible 2012',
 'Aston Martin V8 Vantage Coupe 2012',
 'Aston Martin Virage Convertible 2012',
 'Aston Martin Virage Coupe 2012',
]

!unzip car_data.zip

Archive:  car_data.zip
replace car_data/test/Acura Integra Type R 2001/00128.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [4]:
detector = Detector('yolo')

W0616 15:45:56.330759 139899116312448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0616 15:45:56.334869 139899116312448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0616 15:45:56.339552 139899116312448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0616 15:45:56.807513 139899116312448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0616 15:45:56.808904 139899116312448 

## Crop data

In [6]:
!mkdir cropped_car_test

detector.crop_path(
    '../content/car_data/test/', 
    '../content/cropped_car_test/',
    dirs,
    params={
        'detect_largest_box': True,
        'smallest_detected_area': 0.5,
    }
)

Directory 0 of 196 ...
Images in directory 0 "Audi 100 Sedan 1994" cropped
Directory 1 of 196 ...


ValueError: ignored

## Zip for an output

In [0]:
!zip -r compressed.zip cropped_car_test